In [1]:
from neo4j import GraphDatabase

# Replace with your Aura connection details
URI = "neo4j+s://104abb61.databases.neo4j.io"
USER = "neo4j"
PASSWORD = "uSV92xXhnyT0B2nLnSTOaqna4FfQxJHHbdRmqdE___A"
CSV_PATH1 = "Mock_Rule_Set_1.csv"
CSV_PATH2 = "Mock_Rule_Set_2.csv"

# Connect to Neo4j
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

In [30]:
from openai import OpenAI

client = OpenAI(api_key="sk-097bd9e031674cab87bef5f05e49ed19", base_url="https://api.deepseek.com")


def extract_facts(prompt):
    system_prompt = """
    You extract structured patient info from a sentence. Output JSON with these keys:
    - gender: "Male" or "Female"
    - age: number
    - symptoms: list of strings
    - lab_tests: an OBJECT where each key is a lab test name and each value is the result (e.g., "Hemoglobin": "Low")
    """
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        stream=False
    )
    return response.choices[0].message.content


In [17]:
def expand_symptoms(symptom):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                "role": "system",
                "content": "You are a medical NLP assistant. Given a symptom, return 3 to 5 similar or related medical symptoms as a JSON list. Do not include definitions."
            },
            {
                "role": "user",
                "content": f"Expand this symptom: {symptom}"
            }
        ],
        stream=False
    )
    import re, json
    cleaned = re.sub(r"```json|```", "", response.choices[0].message.content).strip()
    try:
        return json.loads(cleaned)
    except Exception:
        return [symptom]  # fallback

In [18]:
def expand_all_symptoms(symptoms):
    all_expanded = set()
    for s in symptoms:
        expanded = expand_symptoms(s)
        all_expanded.update(expanded)
    return list(all_expanded)

In [4]:
def generate_cypher_query(parsed_json):
    gender = parsed_json["gender"]
    age = parsed_json["age"]
    symptoms = parsed_json["symptoms"]
    labs = parsed_json["lab_tests"]

    symptom_match = "\n".join([
        f"MATCH (s{i}:Symptom {{name: '{s}'}})-[:PART_OF]->(r)" for i, s in enumerate(symptoms)
    ])
    lab_match = "\n".join([
        f"MATCH (l{j}:LabTest {{name: '{name}'}})-[rel{j}:PART_OF {{result: '{res}'}}]->(r)" for j, (name, res) in enumerate(labs.items())
    ])

    return f"""
{symptom_match}
{lab_match}
MATCH (r)-[:IMPLIES]->(d:Diagnosis)
RETURN d.name AS diagnosis, r.rule_id AS rule
"""


In [5]:
def query_neo4j(cypher_string, driver):
    with driver.session() as session:
        result = session.run(cypher_string)
        return [record.data() for record in result]


In [6]:
def get_prescriptions(diagnosis_name, driver):
    cypher = f"""
    MATCH (d:Diagnosis {{name: '{diagnosis_name}'}})
    <-[:IMPLIES]-(r:Rule)
    -[:RECOMMENDS {{type: 'Prescription'}}]->(m:Prescription)
    RETURN DISTINCT m.name AS prescription
    """
    return query_neo4j(cypher, driver)


In [42]:
def generate_final_response(context, prompt):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "The following context contains a diagnosis based on the information patient provided, and prescriptions related to that diagnosis. Give a medical answer based on this context."},
            {"role": "user", "content": f"{prompt}\n\nContext:\n{context}"}
        ],
        stream=False
    )
    return response.choices[0].message.content


In [ ]:
# def rag_diagnosis_pipeline(user_input, driver):
#     # Step 1: Extract structured facts
#     parsed = extract_facts(user_input)
#     print("🔍 Parsed facts:", parsed)

#     import json
#     facts = json.loads(parsed)

#     # Step 2: Generate Cypher
#     cypher_query = generate_cypher_query(facts)
#     print("\n📡 Generated Cypher:\n", cypher_query)

#     # Step 3: Query KG for diagnosis
#     diagnosis_results = query_neo4j(cypher_query, driver)
#     print("\n🧠 Diagnoses Found:", diagnosis_results)

#     # Step 4: Get prescriptions for top result
#     if diagnosis_results:
#         top_diagnosis = diagnosis_results[0]['diagnosis']
#         prescriptions = get_prescriptions(top_diagnosis, driver)
#     else:
#         top_diagnosis = "No strong match found"
#         prescriptions = []

#     # Step 5: Build final context & generate LLM answer
#     context = f"Diagnosis: {top_diagnosis}\nPrescriptions: {[p['prescription'] for p in prescriptions]}"
#     final_answer = generate_final_response(context, user_input)
#     print("\n📝 Final Answer:\n", final_answer)


In [43]:
def generate_fuzzy_cypher_query(gender, age, expanded_symptoms, lab_tests):
    symptom_list_str = ', '.join([f'"{s}"' for s in expanded_symptoms])

    symptom_match = f"""
    OPTIONAL MATCH (s:Symptom)-[:PART_OF]->(r)
    WHERE s.name IN [{symptom_list_str}]
    """

    lab_match = "\n".join([
        f"""OPTIONAL MATCH (l:LabTest {{name: '{name}'}})-[rel{j}:PART_OF {{result: '{res}'}}]->(r)"""
        for j, (name, res) in enumerate(lab_tests.items())
    ])

    patient_profile = ""
    if gender or age:
        profile_clauses = []
        if gender:
            profile_clauses.append(f"p.gender = '{gender}'")
        if age:
            if age >= 60:
                profile_clauses.append("p.age_group = '>60'")
            elif age >= 40:
                profile_clauses.append("p.age_group = '40-60'")
            else:
                profile_clauses.append("p.age_group = '<40'")
        patient_profile = f"""
        OPTIONAL MATCH (p:PatientProfile)-[:PART_OF]->(r)
        WHERE {' AND '.join(profile_clauses)}
        """

    return f"""
MATCH (r:Rule)
{symptom_match}
{lab_match}
{patient_profile}
OPTIONAL MATCH (r)-[:IMPLIES]->(d:Diagnosis)
WITH r, d, count(DISTINCT s) AS symptom_score, count(DISTINCT p) AS profile_score,
     {len(lab_tests)} AS expected_lab_count,
     size([rel IN [{', '.join(f'rel{j}' for j in range(len(lab_tests)))}] WHERE rel IS NOT NULL]) AS lab_score
WHERE symptom_score + lab_score + profile_score > 0
RETURN d.name AS diagnosis, r.rule_id AS rule, symptom_score, lab_score, profile_score,
       symptom_score + lab_score + profile_score AS total_score
ORDER BY total_score DESC
LIMIT 5
"""



In [48]:

user_input = "I'm a 67-year-old male. I feel tired and dizzy. My hemoglobin is low. What might I have?"

# Step 1: Extract patient info
parsed = extract_facts(user_input)
print("🔍 Parsed:", parsed)

import json, re
cleaned = re.findall(r'({(?:.|\n)*})', parsed, re.DOTALL)[0].strip()
print(cleaned)
facts = json.loads(cleaned)

gender = facts.get("gender")
age = facts.get("age")
symptoms = facts.get("symptoms", [])
lab_tests = facts.get("lab_tests", {})

# Step 2: Expand symptoms
expanded_symptoms = expand_all_symptoms(symptoms)
expanded_symptoms = [s.capitalize() for s in expanded_symptoms]
print("🔁 Expanded symptoms:", expanded_symptoms)

# Step 3: Generate fuzzy Cypher
cypher_query = generate_fuzzy_cypher_query(gender, age, expanded_symptoms, lab_tests)
print("\n📡 Cypher:\n", cypher_query)

# Step 4: Query KG
diagnosis_results = query_neo4j(cypher_query, driver)
print("\n🧠 Diagnoses:", diagnosis_results)

if diagnosis_results:
    top_diagnosis = diagnosis_results[0]['diagnosis']
    prescriptions = get_prescriptions(top_diagnosis, driver)
else:
    top_diagnosis = "No strong match found"
    prescriptions = []

# Step 5: Compose context + LLM summary
context = f"Diagnosis: {top_diagnosis}\nPrescriptions: {[p['prescription'] for p in prescriptions]}"

final_answer = generate_final_response(context, user_input)
print("\n📝 Final Answer:\n", final_answer)


🔍 Parsed: ```json
{
  "gender": "Male",
  "age": 67,
  "symptoms": ["tired", "dizzy"],
  "lab_tests": {
    "Hemoglobin": "Low"
  }
}
```
{
  "gender": "Male",
  "age": 67,
  "symptoms": ["tired", "dizzy"],
  "lab_tests": {
    "Hemoglobin": "Low"
  }
}
🔁 Expanded symptoms: ['Faint', 'Weakness', 'Lethargy', 'Unsteady', 'Lightheaded', 'Vertigo', 'Drowsiness', 'Fatigue', 'Woozy', 'Exhaustion']

📡 Cypher:
 
MATCH (r:Rule)

    OPTIONAL MATCH (s:Symptom)-[:PART_OF]->(r)
    WHERE s.name IN ["Faint", "Weakness", "Lethargy", "Unsteady", "Lightheaded", "Vertigo", "Drowsiness", "Fatigue", "Woozy", "Exhaustion"]
    
OPTIONAL MATCH (l:LabTest {name: 'Hemoglobin'})-[rel0:PART_OF {result: 'Low'}]->(r)

        OPTIONAL MATCH (p:PatientProfile)-[:PART_OF]->(r)
        WHERE p.gender = 'Male' AND p.age_group = '>60'
        
OPTIONAL MATCH (r)-[:IMPLIES]->(d:Diagnosis)
WITH r, d, count(DISTINCT s) AS symptom_score, count(DISTINCT p) AS profile_score,
     1 AS expected_lab_count,
     size([rel IN 

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\nMATCH (r:Rule)\n\n    OPTIONAL MATCH (s:Symptom)-[:PART_OF]->(r)\n    WHERE s.name IN ["Faint", "Weakness", "Lethargy", "Unsteady", "Lightheaded", "Vertigo", "Drowsiness", "Fatigue", "Woozy", "Exhaustion"]\n    \nOPTIONAL MATCH (l:LabTest {name: \'Hemoglobin\'})-[rel0:PART_OF {result: \'Low\'}]->(r)\n\n        OPTIONAL MATCH (p:PatientProfile)-[:PART_OF]->(r)\n        WHERE p.gender = \'Male\' AND p.age_group = \'>60\'\n        \nOPTIONAL MATCH (r)-[:IMPLIES]->(d:Diagnosis)\nWITH r, d, count(DISTINCT s) AS symptom_score, count(DISTINCT p) AS profile_score,\n     1 AS expected_lab_count,\n     size([rel IN [rel0] WHERE rel IS NOT NULL]) AS lab_score\nWHERE symptom_score + lab_score 


🧠 Diagnoses: [{'diagnosis': 'Anemia', 'rule': 'R5', 'symptom_score': 1, 'lab_score': 1, 'profile_score': 0, 'total_score': 2}, {'diagnosis': 'Myocardial Infarction', 'rule': 'R1', 'symptom_score': 0, 'lab_score': 0, 'profile_score': 1, 'total_score': 1}]

📝 Final Answer:
 Based on the information provided, you have **anemia**, which is a condition characterized by a low hemoglobin level. Hemoglobin is the protein in red blood cells that carries oxygen throughout your body. When hemoglobin is low, it can lead to symptoms like fatigue, dizziness, weakness, and shortness of breath.

### Possible Causes of Anemia in Your Case:
1. **Iron Deficiency Anemia** – The most common type, often due to poor dietary intake, chronic blood loss (e.g., from gastrointestinal bleeding), or malabsorption.  
2. **Vitamin B12 or Folate Deficiency** – Can occur due to poor diet (especially in vegetarians/vegans), pernicious anemia (B12 absorption issue), or alcoholism.  
3. **Chronic Disease Anemia** – Assoc